# PyRAF: Fully loaded

This notebook illustrates how to find all of the tasks available in the current IRAF/PyRAF distribution.  

In [1]:
from __future__ import print_function
from pyraf import iraf
from stsci.tools.irafglobals import IrafTask, IrafPkg, IrafError
from types import FunctionType
from astropy.table import Table

In [2]:
def load_all_packages():
    """Recursively load all IRAF packages into the current namespace
    
    Parameters
    ----------
    None
    
    Returns
    -------
    errored_packages : dict
        package name and exception for any packages that could not be loaded
    
    """
    
    errored_packages = {}
    
    previous_size = 0
    
    #-- Iterate until no new packages have been loaded
    while len(iraf.mmdict) > previous_size:
        previous_size = len(iraf.mmdict)
       
        for name in list(iraf.mmdict.keys()):
            obj = iraf.mmdict[name]
            
            if isinstance(obj, IrafPkg):
                try:
                    print("Loading {}".format(name))
                    obj()
                except (AttributeError, ImportError, IrafError, EOFError) as e:
                    errored_packages[name] = e
                    print("ERROR WITH: {}".format(name))
                    print("   BECAUSE: {}".format(e.message))
                    
    return errored_packages

In [3]:
def get_full_path(taskname):
    """Assemble full path name for a given IRAF task
    
    Find the task's parent, then each parent's parent until the top-level 
    ("clpackage") is reached.  All parents and the task will be concatenated and
    separated by ".". 
    
    Parameters
    ----------
    taskname : str
        Name of any IRAF task loaded into the namespace
        
    Returns
    -------
    outname : str
        Full path to the input task through the namespace
        
    Examples
    --------
        >>> get_full_path('mscombine')
        'clpackage.stsdas.toolbox.imgtools.mstools.mscombine'
    """
    
    outname = getattr(iraf, taskname).getPkgname() + '.' + taskname
    
    while outname.split('.')[0] != 'clpackage':
        outname = getattr(iraf, outname.split('.')[0]).getPkgname() + '.' + outname
        
    return outname

In [4]:
def find_all_tasks():
    """Find all tasks loaded into the current namespace
    
    Ignoring built-in functions, assemble the sorted list
    of all currently available IRAF tasks.
    
    Parameters
    ----------
    None
    
    Returns
    -------
    all_tasks : list
        Full path of all currently available tasks.
    """
    
    all_tasks = []
    for name in list(iraf.mmdict.keys()):
        obj = iraf.mmdict[name]
            
        #-- cannot check only if instance of IrafTask, as an IrafTask
        #-- also are subclasses of IrafPkg.
        if not isinstance(obj, IrafPkg) and isinstance(obj, IrafTask):
            #print get_full_path(name)
            all_tasks.append(get_full_path(name))
            
    return sorted(all_tasks)

## Loading every package and sub-package we can find

A single call to `load_all_packages()` will continuously load every package and sub-package it can find untill no more sub-packages have been added to the namespace.  It will also return any failed packages, so that we can diagnose any failures.

In [5]:
failed = load_all_packages()

Loading nfextern
nfextern/:
 ace/           msctools/       newfirm/        odi/
Loading tv
Loading imfit
Loading song
ERROR WITH: song
   BECAUSE: Undefined IRAF task `rvx'
Loading obsolete
obsolete/:
 imtitle        ofixpix         oimstatistics   owfits
 mkhistogram    oimcombine      orfits          radplt
Loading guiapps
guiapps/:
 demo           spt/            xapphot/        xhelp           xrv/
Loading cfh12k
This is the initial release of the IRAF FITSUTIL package
to include support for FITS tile compression via 'fpack'.
Please send comments and questions to seaman@noao.edu.

cfh12k/:
 hdrcfh12k      setcfh12k
Loading nttools
Loading iis
iis/:
 blink          erase           monochrome      window
 cv             frame           pseudocolor     zoom
 cvl            lumatch         rgb
Loading plot
Loading rvsao

#-----------------------------------------------------------+
#           RVSAO Radial Velocity Analysis Package          |
#           Smithsonian Astrophysical Obse

/Users/ely/anaconda3/envs/iraf/lib/python2.7/site-packages/ipykernel/__main__.py:33: DeprecationWarning: BaseException.message has been deprecated as of Python 2.6


ERROR WITH: kepler
   BECAUSE: No module named scipy
Loading mscdb
ERROR WITH: mscdb
   BECAUSE: Cannot find executable for task mscdb
Tried /Users/ely/anaconda3/envs/iraf/iraf/bin.macosx/mscdb.cl, /Users/ely/anaconda3/envs/iraf/variants/common//iraf/mscdb/mscdb.cl
Loading upsqiid
upsqiid/:
 chlist         imgraph         movproc         sqsections      usqremap
 chorient       imlinfit        nircombine      statelist       usqsky
 cleanup        imlinregress    notchlist       stdproc         where
 closure        imparse         overlap         stdreport       which
 expandnim      imquadfit       patproc         temp_plot       xyadopt
 filedir        imzero          photproc        tmove           xyget
 fileroot       iterstat        pltnaac         transmat        xylap
 getcenters     linklaps        pltstat         usqcorr         xytrace
 getmap         locate          proctest        usqdark         zget
 getstar        mergecom        rechannel       usqflat         ztrace


In [6]:
print("Found {} failed packages.".format(len(failed)))
print("-"*20)
for item in failed:
    print(item, ' ->', failed[item])

Found 13 failed packages.
--------------------
xobsolete  -> Cannot find executable for task xobsolete
Tried /Users/ely/anaconda3/envs/iraf/variants/common//iraf/xray/bin.macosx/xobsolete.cl, /Users/ely/anaconda3/envs/iraf/variants/common//iraf/xray/xobsolete/xobsolete.cl
song  -> Undefined IRAF task `rvx'
wfpc  -> No module named wfpc2tools
sobsolete  -> Cannot find executable for task sobsolete
Tried /Users/ely/anaconda3/envs/iraf/variants/common//iraf/stsci_iraf//stsdas/bin/sobsolete.cl, /Users/ely/anaconda3/envs/iraf/variants/common//iraf/stsci_iraf//stsdas/pkg/sobsolete/sobsolete.cl
mscdb  -> Cannot find executable for task mscdb
Tried /Users/ely/anaconda3/envs/iraf/iraf/bin.macosx/mscdb.cl, /Users/ely/anaconda3/envs/iraf/variants/common//iraf/mscdb/mscdb.cl
apdemos  -> EOF on parameter prompt
kepler  -> No module named scipy
xlocal  -> Cannot find executable for task xlocal
Tried /Users/ely/anaconda3/envs/iraf/variants/common//iraf/xray/bin.macosx/xlocal.cl, /Users/ely/anaconda3/

## After all packages have been loaded, now we need to identify every task

In [7]:
every_loaded_task = find_all_tasks()
print("Found {} loaded tasks.".format(len(every_loaded_task)))

Found 2160 loaded tasks.


In [8]:
for item in every_loaded_task:
    print(item)

clpackage._logout
clpackage.adccdrom.catalog
clpackage.adccdrom.spectra
clpackage.adccdrom.tbldb
clpackage.cfh12k.hdrcfh12k
clpackage.cfh12k.setcfh12k
clpackage.cirred.atmo_cor
clpackage.cirred.calc_off
clpackage.cirred.clearim
clpackage.cirred.do_ccmap
clpackage.cirred.do_osiris
clpackage.cirred.do_wcs
clpackage.cirred.extra
clpackage.cirred.fixbad
clpackage.cirred.fixfits
clpackage.cirred.irdiff
clpackage.cirred.maskbad
clpackage.cirred.med
clpackage.cirred.osiris
clpackage.cirred.shift_comb
clpackage.cirred.sky_sub
clpackage.cirred.spec_comb
clpackage.cl
clpackage.ctio.apropos
clpackage.ctio.bin2iraf
clpackage.ctio.bitstat
clpackage.ctio.chpixfile
clpackage.ctio.colselect
clpackage.ctio.compairmass
clpackage.ctio.compression._compress
clpackage.ctio.compression._uncompress
clpackage.ctio.compression.fitsread
clpackage.ctio.compression.fitswrite
clpackage.ctio.compression.imcompress
clpackage.ctio.compression.improc
clpackage.ctio.compression.imuncompress
clpackage.ctio.coords
clpack

### Just for a prettier view

In [9]:
max_size = 0
for item in every_loaded_task:
    size = len(item.split('.')[1:])
    
    if size > max_size:
        max_size = size
    

all_rows = []
for item in every_loaded_task:
    split_path = item.split('.')[1:]
    new_row = [item for item in split_path[:-1]] + ['' for i in range(max_size - len(split_path[:-1]) -1)] + [split_path[-1]]
    all_rows.append(tuple(new_row))
    
    
names = tuple(['base'] + ['sub{}'.format(i) for i in range(max_size - 2)] + ['task'])
dtype = ['S10'] * max_size

t = Table(rows=all_rows, names=names, dtype=dtype)

In [10]:
t.show_in_notebook()

idx,base,sub0,sub1,sub2,task
0,,,,,_logout
1,adccdrom,,,,catalog
2,adccdrom,,,,spectra
3,adccdrom,,,,tbldb
4,cfh12k,,,,hdrcfh12k
5,cfh12k,,,,setcfh12k
6,cirred,,,,atmo_cor
7,cirred,,,,calc_off
8,cirred,,,,clearim
9,cirred,,,,do_ccmap
